In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
imdbreviews=pd.read_csv("/content/IMDB Dataset.csv")

In [ ]:
imdbreviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [ ]:
imdbreviews['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [ ]:
# Dependent Variable(y) is sentiment. It is non numeric and binary.
# Classification models to be built. Binary Logistic regression is classification
# algorithm widely used

# Binary Logistic regression explains relationship between a binary dependent
# variable(y) and multiple independent variables(X's).
# Binary Logistic Regression outcome is an equation
#       e^B0+B1X1+B2X2+B3X3+...........+BnXn                          1
#. p = ----------------------------------------or Sigmoid = -------------------------
#.      1+ e^B0+B1X1+B2X2+B3X3+...........+BnXn            1+e^-(B0+B1X1+B2X2..BnXn)
# p is probability.  e is exponential or 2.718 . B0 is intercept
# B1,B2, B3,...Bn - Coefficients. X1,X2,X3,...Xn - Independent Variables

# Classification based on predicted probability
# probability greater than equal to 0.50 ---- 1 or Yes or True
# probability less than 0.50 ------ 0.  or No or False

# Assumptions are:
# a) dependent variable(y) must be binary
# b) Independent Variables(X) can be both numerical and non numerical
# c) There must be logical relationship between dependent variable and independent
# variables
# d) No Multicollinearity
# e) Sample size required is minimum 50 observations per variable

# Interpretation of output based on Confusion matrix
# Confusion Matrix is a 2 X 2 matrix or cross tabulation of actual and predicted
# class
# Primary metric is Accuracy of model which must be between 0.70 - 0.95
#                         True Positive + True Negative
# Accuracy = ------------------------------------------------------------------
#             True Positive + False Negative + False Positive + True Negative

# Other metrics that are calculated from confusion matrix are:-
# Sensitivity or Recall = TP/(TP+FN) . how many positives predicted positive
# Specificty = TN/(TN+FP). how many negatives predicted negative
# Precision = TP/(TP+FP). Overall positive prediction rate.
# F1 Score = 2 X ((Precision X Recall)/(Precision + Recall))

In [ ]:
imdbreviews['review'].head(3) # Text data must be converted to matrix of numbers

,review
0,One of the other reviewers has mentioned that ...
1,A wonderful little production. <br /><br />The...
2,I thought this was a wonderful way to spend ti...


In [ ]:
imdbreviews['review']=imdbreviews['review'].str.lower()

In [ ]:
imdbreviews['review']=imdbreviews['review'].str.replace(r'a-zA-Z0-9\w\s.','')

In [ ]:
imdbreviews['review']=imdbreviews['review'].str.replace(r'<br />','')

In [ ]:
imdbreviews['review']=imdbreviews['review'].str.replace(r'\d+','',regex=True)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf=TfidfVectorizer(max_features=300,stop_words="english",ngram_range=(2,2))

In [ ]:
X_tfidf=tfidf.fit_transform(imdbreviews['review'])

In [ ]:
X_tfidf=pd.DataFrame(X_tfidf.toarray(),columns=tfidf.get_feature_names_out())

In [ ]:
X_tfidf.head(2)

,acting bad,acting good,action movie,action scenes,action sequences,bad acting,bad bad,bad film,bad good,bad guy,...,worth seeing,worth watching,writer director,year old,years ago,years later,years old,young girl,young man,young woman
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from sklearn.preprocessing import LabelEncoder

The LabelEncoder is used to convert categorical text labels into numerical format. In this notebook, the 'sentiment' column contains string values ('positive' and 'negative'). Machine learning models, such as the Logistic Regression model being built here, generally require numerical input. LabelEncoder transforms these categorical labels into integers, typically 0 and 1, which the model can then process.

In [ ]:
y=LabelEncoder().fit_transform(imdbreviews['sentiment'])

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logistic_model=LogisticRegression().fit(X_tfidf,y)

In [ ]:
logistic_model.score(X_tfidf,y)

0.6961

In [ ]:
logistic_predict=logistic_model.predict(X_tfidf)

In [ ]:
pd.crosstab(y,logistic_predict)
# 0 - Negative  & 1 - Positive
# True Negative-15508,False Positive-9492,False Negative-5703,True Positive-19297

col_0,0,1
row_0,,
0,15508,9492
1,5703,19297


In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y,logistic_predict))

              precision    recall  f1-score   support

           0       0.73      0.62      0.67     25000
           1       0.67      0.77      0.72     25000

    accuracy                           0.70     50000
   macro avg       0.70      0.70      0.69     50000
weighted avg       0.70      0.70      0.69     50000



In [ ]:
!pip install streamlit
import streamlit as st
import numpy as np
import joblib

# --------------------------------------------------
# Load your trained model and vectorizer
# (Adjust these paths/names to match your file)
# --------------------------------------------------

# Streamlit UI
st.title("🎬 IMDB Review Sentiment Analyzer")
st.write("Enter a movie review below and the model will predict its sentiment.")

# Text input box
user_review = st.text_area("Write a review:", height=200)

# Predict button
if st.button("Analyze Sentiment"):
    if user_review.strip() == "":
        st.warning("Please enter a review.")
    else:
        # Transform the text
        X_new = tfidf.transform([user_review])

        # Predict sentiment
        prediction = logistic_model.predict(X_new)[0]

        # Predict probability
        prob = logistic_model.predict_proba(X_new)[0]
        pos_prob = prob[1]
        neg_prob = prob[0]

        # Display results
        st.subheader("Sentiment Result")
        sentiment_text = "👍 Positive" if prediction == 1 else "👎 Negative"
        st.write(f"**Prediction:** {sentiment_text}")

        st.subheader("Probability Scores")
        st.write(f"**Positive Probability:** {pos_prob:.4f}")
        st.write(f"**Negative Probability:** {neg_prob:.4f}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 110.1 MB/s eta 0:00:00


2025-12-07 06:03:21.475 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 06:03:21.687 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-12-07 06:03:21.688 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 06:03:21.691 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 06:03:21.692 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 06:03:21.694 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 06:03:21.695 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 06:03:21.697 Thread 'MainThread': mi